In [ ]:
import torchaudio
import torch
import numpy as np
# from datasets import Dataset
from transformers import WhisperFeatureExtractor
import torch
from datasets import Dataset
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    # DataCollatorSpeechSeq2SeqWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

raw_path = "/ocean/projects/cis250085p/shared/A_track/"

torch.cuda.is_available()


/ocean/projects/cis250085p/shared/envPreproces/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
torch.cuda.is_available()

True

In [ ]:
# --- All the setup from before ---
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
processor.tokenizer.set_prefix_tokens(language="kinyarwanda", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16,cache_dir = "/ocean/projects/cis250085p/shared/A_track") # Load in fp16 to save memory


In [15]:
%cd /ocean/projects/cis250085p/shared/A_track

/ocean/projects/cis250085p/shared/A_track


In [35]:
from transformers import Seq2SeqTrainer
import datetime

# Give it a completely new name to avoid any caching issues.
class FinalDebuggingTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        This is the definitive version with a print statement for debugging.
        If we see the printout, we know this code is running.
        """
        # --- DEBUGGING PRINT STATEMENT ---
        # print(f"--- EXECUTING FinalDebuggingTrainer.compute_loss at {datetime.datetime.now()} ---")

        # The rest of the logic remains the same.
        outputs = model(**inputs)
        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        return (loss, outputs) if return_outputs else loss

In [ ]:
# Configure LoRA
# model = prepare_model_for_kbit_training(model)  # <-- COMMENT OUT OR DELETE THIS

model = adapt_model_to_80_bins(model)


# 3. CONFIGURE LORA (on the newly adapted model)
# You can now proceed with your LoRA setup as before
config = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")
model = get_peft_model(model, config)

model.print_trainable_parameters()


data_collator = robust_data_collator
# Define Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3-lora-streaming",
    # output_dir="/ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-6000",
    per_device_train_batch_size=2,  # Adjust based on your VRAM
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    warmup_steps=100,
    num_train_epochs=5,
    fp16=True, # Already loaded model in fp16, but this ensures trainer uses it
    logging_steps=100,
    save_strategy="steps",
    save_steps=1000,
    # --- Important for streaming ---
    # The dataloader will now be much more efficient
    dataloader_num_workers=4, # Use multiple CPU cores to load data in parallel
    remove_unused_columns=False,


        # --- NEW ARGUMENTS FOR EVALUATION ---
    do_eval=True,
    eval_steps=1000,                       # Evaluate every 1000 steps
    # load_best_model_at_end=True,          # Load the best model (lowest WER) at the end of training
    metric_for_best_model="wer",          # The metric to monitor for the best model
    greater_is_better=False,             # For WER, a lower score is better
    predict_with_generate=True,           # **CRITICAL**: This tells the Trainer to use the .generate() method for inference, which is required for Whisper.

)

trainer = FinalDebuggingTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset,
    data_collator=data_collator,
    # The 'tokenizer' argument is deprecated, but we'll leave it for now
    # as the warning is not critical to this error.
    tokenizer=processor.feature_extractor,

    eval_dataset=eval_dataset,          # <-- Pass the evaluation dataset
    compute_metrics=compute_metrics,    # <-- Pass your metrics function
    

)
# Start Fine-Tuning
print("Starting memory-efficient LoRA fine-tuning...")
trainer.train(

    # resume_from_checkpoint=True,
    resume_from_checkpoint="/ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-5000",
)

print("Saving final LoRA adapter...")
trainer.save_model("./whisper-large-v3-lora-streaming-final")

Adapting the model to accept 80-channel input...
✅ Model successfully adapted.


/ocean/projects/cis250085p/shared/envPreproces/lib/python3.11/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/ocean/projects/cis250085p/shared/envPreproces/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


/var/tmp/ipykernel_49223/2288983708.py:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FinalDebuggingTrainer.__init__`. Use `processing_class` instead.
  trainer = FinalDebuggingTrainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 3,932,160 || all params: 1,547,238,400 || trainable%: 0.2541
Starting memory-efficient LoRA fine-tuning...


Step,Training Loss
5100,5.308700
5200,3.150300
5300,2.645300


KeyboardInterrupt: 

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel, LoraConfig
import torch

# (Your 'adapt_model_to_80_bins' function should be defined here)
# (Your 'robust_data_collator' function should be defined here)
# (Your 'FinalDebuggingTrainer' class should be defined here)

# --- 1. Load the Base Model ---
model_name = "openai/whisper-large-v3"
print(f"Loading base model: {model_name}")
processor = WhisperProcessor.from_pretrained(model_name)
processor.tokenizer.set_prefix_tokens(language="kinyarwanda", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16,cache_dir = "/ocean/projects/cis250085p/shared/A_track") # Load in fp16 to save memory

# --- 2. Perform the Model Surgery ---
model = adapt_model_to_80_bins(model)

# --- 3. Manually Load Your Trained LoRA Adapter ---
# This is the key step. We are loading the progress from your best checkpoint.
checkpoint_path = "/ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-6000"
print(f"Manually loading LoRA adapter from: {checkpoint_path}")
model = PeftModel.from_pretrained(model, checkpoint_path, is_trainable=True)

print("\nModel and adapter loaded successfully.")
model.print_trainable_parameters()

Loading base model: openai/whisper-large-v3


Adapting the model to accept 80-channel input...
✅ Model successfully adapted.
Manually loading LoRA adapter from: /ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-6000

Model and adapter loaded successfully.
trainable params: 3,932,160 || all params: 1,547,238,400 || trainable%: 0.2541


/ocean/projects/cis250085p/shared/envPreproces/lib/python3.11/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.2.self_attn.v_proj.lora_B.default.weight', 'base_mod

In [ ]:

# data_collator = robust_data_collator
# # Define Training Arguments
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-large-v3-lora-streaming",
#     # output_dir="/ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-6000",
#     per_device_train_batch_size=2,  # Adjust based on your VRAM
#     gradient_accumulation_steps=2,
#     learning_rate=1e-4,
#     warmup_steps=100,
#     num_train_epochs=5,
#     fp16=True, # Already loaded model in fp16, but this ensures trainer uses it
#     logging_steps=100,
#     save_strategy="steps",
#     save_steps=1000,
#     # --- Important for streaming ---
#     # The dataloader will now be much more efficient
#     dataloader_num_workers=4, # Use multiple CPU cores to load data in parallel
#     remove_unused_columns=False,


#         # --- NEW ARGUMENTS FOR EVALUATION ---
#     do_eval=True,
#     eval_steps=100,                       # Evaluate every 1000 steps
#     # load_best_model_at_end=True,          # Load the best model (lowest WER) at the end of training
#     metric_for_best_model="wer",          # The metric to monitor for the best model
#     greater_is_better=False,             # For WER, a lower score is better
#     predict_with_generate=True,           # **CRITICAL**: This tells the Trainer to use the .generate() method for inference, which is required for Whisper.

# )

# # Initialize Trainer
# # trainer = CustomSeq2SeqTrainer(
# #     args=training_args,
# #     model=model,
# #     train_dataset=dataset, # Pass the transformed dataset
# #     data_collator=data_collator,
# #     tokenizer=processor.feature_extractor,
# # )

# trainer = FinalDebuggingTrainer(
#     args=training_args,
#     model=model,
#     train_dataset=dataset,
#     data_collator=data_collator,
#     # The 'tokenizer' argument is deprecated, but we'll leave it for now
#     # as the warning is not critical to this error.
#     tokenizer=processor.feature_extractor,

#     eval_dataset=eval_dataset,          # <-- Pass the evaluation dataset
#     compute_metrics=compute_metrics,    # <-- Pass your metrics function
    

# )
# # Start Fine-Tuning
# print("Starting memory-efficient LoRA fine-tuning...")
# trainer.train(
# )

# print("Saving final LoRA adapter...")
# trainer.save_model("./whisper-large-v3-lora-streaming-final")

/var/tmp/ipykernel_49223/3232425285.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FinalDebuggingTrainer.__init__`. Use `processing_class` instead.
  trainer = FinalDebuggingTrainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting memory-efficient LoRA fine-tuning...


Step,Training Loss


KeyboardInterrupt: 

In [70]:
import torch
import warnings
from pickle import UnpicklingError
import numpy as np # It's good practice to import numpy

def robust_data_collator(features):
    """
    This is the final, fully corrected version. It handles corrupted files AND
    correctly handles the case where the saved files are NumPy arrays.
    """
    valid_features = []
    
    # Part 1: Transformation
    for feature in features:
        try:
            # 1. Load the object from disk (which we now know is a numpy array)
            numpy_array = torch.load(raw_path + feature["file_path"], weights_only=False)
            
            # --- THE FIX: First convert numpy array to torch tensor, THEN change dtype ---
            audio_tensor = torch.from_numpy(numpy_array).to(torch.float16)
            
            # 2. Tokenize the transcription
            tokenized_label = processor.tokenizer(feature["transcription"]).input_ids
            valid_features.append({
                "input_features": audio_tensor,
                "labels": tokenized_label
            })
            
        except (EOFError, RuntimeError, UnpicklingError) as e:
            warnings.warn(f"Skipping corrupted file: {feature['file_path']} | Error: {e}")
            continue

    # Part 2: Padding (only on the valid features)
    if not valid_features:
        warnings.warn("An entire batch of data was corrupted and has been skipped.")
        return {}
        
    input_features_list = [{"input_features": feature["input_features"]} for feature in valid_features]
    label_features_list = [{"input_ids": feature["labels"]} for feature in valid_features]

    batch = processor.feature_extractor.pad(input_features_list, return_tensors="pt")
    labels_batch = processor.tokenizer.pad(label_features_list, return_tensors="pt")
    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

    if (labels[:, 0] == processor.tokenizer.bos_token_id).all().cpu().item():
        labels = labels[:, 1:]

    batch["labels"] = labels
    
    return batch

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel, LoraConfig
import torch

# (Your 'adapt_model_to_80_bins' function should be defined here)
# (Your 'robust_data_collator' function should be defined here)
# (Your 'FinalDebuggingTrainer' class should be defined here)

# --- 1. Load the Base Model ---
model_name = "openai/whisper-large-v3"
print(f"Loading base model: {model_name}")
processor = WhisperProcessor.from_pretrained(model_name)
processor.tokenizer.set_prefix_tokens(language="kinyarwanda", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16,cache_dir = "/ocean/projects/cis250085p/shared/A_track") # Load in fp16 to save memory

# --- 2. Perform the Model Surgery ---
model = adapt_model_to_80_bins(model)

# This might take a minute the first time it's called
model = torch.compile(model) 

# --- 3. Manually Load Your Trained LoRA Adapter ---
# This is the key step. We are loading the progress from your best checkpoint.
checkpoint_path = "/ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-6000"
print(f"Manually loading LoRA adapter from: {checkpoint_path}")
model = PeftModel.from_pretrained(model, checkpoint_path, is_trainable=True)

# --- 2. THIS IS THE NEW PART: OPTIMIZE GENERATION CONFIG ---
print("Optimizing generation config for faster evaluation...")
model.generation_config.num_beams = 1
model.generation_config.do_sample = False

print("\nModel and adapter loaded successfully.")
model.print_trainable_parameters()

Loading base model: openai/whisper-large-v3


Adapting the model to accept 80-channel input...
✅ Model successfully adapted.
Manually loading LoRA adapter from: /ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-6000
Optimizing generation config for faster evaluation...

Model and adapter loaded successfully.
trainable params: 3,932,160 || all params: 1,547,238,400 || trainable%: 0.2541


/ocean/projects/cis250085p/shared/envPreproces/lib/python3.11/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model._orig_mod.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model._orig_mod.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model._orig_mod.model.encoder.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model._orig_mod.model.encoder.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model._orig_mod.model.encoder.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model._orig_mod.model.encoder.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model._orig_mod.model.encoder.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model._orig_mod.model.encoder.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model._orig_mod.model.encoder.layers.2.self_attn.v_proj.lora_A.default.weight', '

In [ ]:
# --- 4. Define Training Arguments for the New Run ---
training_args = Seq2SeqTrainingArguments(
    # Use a new directory for this resumed run to avoid conflicts
    output_dir="./whisper-large-v3-lora-resumed-run", 
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=1e-4, # You can optionally lower this (e.g., 5e-5) since you're already in a good spot
    warmup_steps=100,
    num_train_epochs=5, # This will train for 5 *additional* epochs
    fp16=True,
    logging_steps=100,
    save_strategy="steps",
    save_steps=1000,

    report_to="wandb",

    remove_unused_columns=False,


    # --- NEW ARGUMENTS FOR EVALUATION ---
    eval_strategy="steps",
    do_eval=True,
    eval_steps=50,                       # Evaluate every 1000 steps
    # load_best_model_at_end=True,          # Load the best model (lowest WER) at the end of training
    metric_for_best_model="wer",          # The metric to monitor for the best model
    greater_is_better=False,             # For WER, a lower score is better
    predict_with_generate=True,           # **CRITICAL**: This tells the Trainer to use the .generate() method for inference, which is required for Whisper.
    generation_max_length=225,
    # generation_num_beams=1,              # Use beam search for better quality
    dataloader_num_workers=5,             # Use multiple CPU cores to load data in parallel
    per_device_eval_batch_size=16,

)

# --- 5. Initialize Trainer ---
trainer = FinalDebuggingTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    data_collator=robust_data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# --- 6. Start the NEW Training Run ---
# We do NOT use resume_from_checkpoint here.
print("Starting a new training run with pre-loaded adapter weights...")
trainer.train()

print("\n✅ Training complete!")
trainer.save_model("./whisper-large-v3-final-model")

/var/tmp/ipykernel_49223/3735704561.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FinalDebuggingTrainer.__init__`. Use `processing_class` instead.
  trainer = FinalDebuggingTrainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting a new training run with pre-loaded adapter weights...


Step,Training Loss,Validation Loss


In [60]:
# !pip install evaluate

In [1]:
import torch
import evaluate
import pandas as pd
from datasets import Dataset, Audio
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel
from torch.utils.data import DataLoader
from tqdm import tqdm


/ocean/projects/cis250085p/shared/envPreproces/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:

# ===================================================================================
# --- Step 1: Configuration - UPDATE THESE PATHS ---
# ===================================================================================
# The original model you fine-tuned
MODEL_NAME = "openai/whisper-large-v3"

# Path to the LoRA adapter you trained and saved
# This is the directory created by `trainer.save_model()`
ADAPTER_PATH = "/ocean/projects/cis250085p/shared/A_track/whisper-large-v3-lora-streaming/checkpoint-4000"

# Path to your test data metadata file
TEST_DATA_CSV = "/ocean/projects/cis250085p/shared/A_track/dev_test.json" # <-- IMPORTANT: Change this!

# The base path where your raw audio files are stored, same as in training
RAW_AUDIO_BASE_PATH = "/ocean/projects/cis250085p/shared/A_track/"

# Configuration for the prediction run
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2 # Adjust based on your GPU memory


In [ ]:

# ===================================================================================
# --- Step 2: Load Your Fine-Tuned Model and Processor ---
# ===================================================================================
print("Loading model and processor...")

# Load the processor
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

processor.tokenizer.set_prefix_tokens(language="kinyarwanda", task="transcribe")

# Load the base model in float16 for faster inference
base_model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    cache_dir = "/ocean/projects/cis250085p/shared/A_track"
).to(DEVICE)


# model = PeftModel.from_pretrained(base_model, ADAPTER_PATH).to(DEVICE)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH, is_trainable=True)

# model.enable_adapter() 

model.eval() # Set the model to evaluation mode

base_model.eval()
# --- OPTIONAL BUT RECOMMENDED: Apply optimizations from training ---
# 1. Use greedy search for maximum speed
# model.generation_config.num_beams = 1
# model.generation_config.do_sample = False
# 2. Compile the model if using PyTorch 2.0+
# model = torch.compile(model)

print("✅ Model loaded and configured for inference.")


Loading model and processor...


AttributeError: 'WhisperForConditionalGeneration' object has no attribute 'enable_adapter'

In [48]:
import torch

# Path to your saved adapter weights file
adapter_weights_path = "./A_track/whisper-large-v3-lora-streaming/checkpoint-4000/adapter_model.safetensors" 

print(f"Loading weights from: {adapter_weights_path}")
adapter_weights = torch.load(adapter_weights_path, weights_only=False)

# Let's inspect a key from one of the deeper layers in the decoder
# This key should have been trained and have non-zero values
key_to_check = "base_model.model.model.decoder.layers.15.self_attn.q_proj.lora_A.default.weight"

if key_to_check in adapter_weights:
    weights = adapter_weights[key_to_check]
    print(f"\nSuccessfully found key: {key_to_check}")
    print("Shape of weights tensor:", weights.shape)
    print("A small sample of the weights:")
    print(weights)
    print(f"\nMean of absolute values: {weights.abs().float().mean()}")
else:
    print(f"\n❌ ERROR: Could not find the key '{key_to_check}' in the adapter file.")
    print("This indicates a serious problem with the saved checkpoint.")

Loading weights from: ./A_track/whisper-large-v3-lora-streaming/checkpoint-4000/adapter_model.safetensors


UnpicklingError: invalid load key, '\xec'.

In [45]:
!pwd

/ocean/projects/cis250085p/shared


In [4]:
TD_df= pd.read_json(TEST_DATA_CSV).T
# TD_df["file_path"] = "processed/"+ TD_df["audio_path"] +".mel.pt"

In [5]:
TD_df["audio_path"] = TD_df.audio_path.str.replace("audio/","/ocean/projects/cis250085p/shared/track_a_audio_files/")  +".wav"

In [6]:
TD_df.audio_path.iloc[0]

'/ocean/projects/cis250085p/shared/track_a_audio_files/1739532284-OogTF7X5UsTPNsR9q4GLZYcJiKB2.wav'

In [7]:
import torchaudio

In [ ]:

# ===================================================================================
# --- Step 3: Load and Prepare the Test Dataset ---
# ===================================================================================
print(f"Loading test dataset from: {TEST_DATA_CSV}")

# Load the metadata
test_dataset = Dataset.from_pandas(TD_df)

# Function to prepare a single example for the model
# It loads the audio and converts it into the 'input_features' the model expects
def prepare_dataset(example):
    audio_path = example["audio_path"]
    try:
        # --- THIS IS THE NEW, MORE ROBUST METHOD ---
        # 1. Load audio directly with torchaudio
        waveform, sample_rate = torchaudio.load(audio_path)

        # 2. The Whisper processor will handle resampling to 16kHz automatically
        # We pass the raw waveform and its original sample rate
        input_features = processor(
            waveform.squeeze(), # Remove channel dimension
            sampling_rate=sample_rate
        ).input_features[0]
        # --- END OF NEW METHOD ---

        # 3. Convert to float16 to match the model
        example["input_features"] = torch.from_numpy(input_features).to(torch.float16)

    except Exception as e:
        # This will now catch any errors from torchaudio or the processor
        print(f"Error processing {audio_path}: {e}")
        example["input_features"] = None # Mark as None to filter later
        
    return example

# Apply the preparation function
test_dataset = test_dataset.map(prepare_dataset, num_proc=4)
# Filter out any samples that failed to load
# test_dataset = test_dataset.filter(lambda example: example["input_features"] is not None)

print(f"✅ Test dataset prepared with {len(test_dataset)} samples.")


Loading test dataset from: /ocean/projects/cis250085p/shared/A_track/dev_test.json


Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4632/4632 [13:30<00:00,  5.72 examples/s]

✅ Test dataset prepared with 4632 samples.


In [38]:

# ===================================================================================
# --- Step 4: Run the Prediction Loop ---
# ===================================================================================
# The data collator just needs to organize the batch
def collate_fn(features):
    input_features = [{"input_features": feature["input_features"]} for feature in features]
    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")
    return batch

# Create a DataLoader for efficient batching
test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

predictions = []
references = test_dataset["transcription"] # Get all ground truth transcriptions

print("Running predictions on the test set...")
# Loop through the test data with a progress bar
for batch in tqdm(test_dataloader):
    # Move batch to the GPU
    inputs = batch["input_features"].to(DEVICE)

    # Run prediction
    with torch.no_grad(): # Disable gradient calculation for inference
        predicted_ids = model.generate(inputs)

    # Decode the predicted IDs to text
    transcriptions = processor.batch_decode(predicted_ids, skip_special_tokens=True, normalize=True)
    predictions.extend(transcriptions)

print("✅ Prediction loop complete.")


Running predictions on the test set...


  0%|                                                                                                                                                                          | 0/290 [00:00<?, ?it/s]

  1%|█                                                                                                                                                               | 2/290 [00:45<1:48:36, 22.63s/it]


KeyboardInterrupt: 

In [24]:
references[:32]

["Akajerekani gateretse hasi ku butaka gapfundikiye neza, gapfundiki umuvuniko w'umweru n'akajerekani karasa n'umweru, akajerekani karimo amata.",
 'Uburyo bwo kwishyura hakoreshejwe ikoranabuhanga, kubitsa, kubikuza, kohereza amafaranga, ukoresheje terefone ngendanwa.',
 "Hano hari abantu batatu babiri muri bo ni ab'igitsina gabo, undi umwe usigaye ni uw'igitsina gore, bicaye ku ntebe z'ibara ry'umweru ndetse imbere yabo bari kunywa amata mu birahure.",
 "Abantu benshi bahagaze imbere y'inyubako ifite amarangi y'umutuku n'umweru, yanditseho amagambo ari mu rurimi rw'icyongereza ari mu ibara ry'umutuku, abagabo barimo bambaye imyenda itandukanye, hari bambaye amashati y'umweru ndetse n'amapantaro y'umukara, hari n'abambaye amakote na karavate, hari n'uwambaye ingofero na rinete.",
 "Ahantu heza hafite isuku ndetse n'amacumbi wasohokera ukaruhuka, hari amazi meza akorerwamo imyidagaduro ndetse na siporo ziruhura imiruho.",
 "Abantu batwara amapikipiki by'umwuga bahetse abagore ku mapiki

In [40]:
predictions

['akajerekani ketereze aasi kutaka kepfundi tieneza kepfundi tiju mufundiko u nyeru na akajerekani klasa na u nyeru akajerekani karimo amata',
 'uwurijobu kweishira hakureishishge ikorana wuhanga kubi ita kubi ikuza kuhiriza ama faranga ukureishishge telefonu jendan',
 'hano hari awa nu batatu babiri muri vo na vijitsi nagavo undi ume usigaye nu vijitsi nagori bicha iku newe zivarariji ume eru ndete imbere ya vo barikungwa amata mwira huri',
 'aba anu benshi wa gaze imbere iinyu baku fite ya maranji yumu tuku nu mngeru ya anditseo ama gamba rimu rimu ligu icho njereza harimu ngibara li jumu tuku aba gawa warimo wa mbae imienda ita ndokanya rabamba ya mashati yu mngeru dete na mapano yumu kaina mbae ama kote na klavate hainu wa mbae ingofero na rinete',
 'anoheza haftisukumdece na machombe wasoche rao karuhu kari amazmeza kwerikomu imetafatru ndetze na sporozi ruhura umuliza',
 'aba anu watuara ama pichi pichi pichumunga wa heze abagode kuma pichi pichi yao vose waagaze iruhande wa hind

In [39]:

# ===================================================================================
# --- Step 5: Calculate and Display the Final WER ---
# ===================================================================================
print("Calculating final Word Error Rate (WER)...")
wer_metric = evaluate.load("wer")

final_wer = wer_metric.compute(predictions=predictions, references=references[:32])

print(f"\n🎉 Final Test WER: {final_wer:.4f} 🎉")


Calculating final Word Error Rate (WER)...

🎉 Final Test WER: 1.4067 🎉


In [30]:

# ===================================================================================
# --- Step 5: Calculate and Display the Final WER ---
# ===================================================================================
print("Calculating final Word Error Rate (WER)...")
wer_metric = evaluate.load("wer")

final_wer = wer_metric.compute(predictions=predictions, references=references[:32])

print(f"\n🎉 Final Test WER: {final_wer:.4f} 🎉")


Calculating final Word Error Rate (WER)...

🎉 Final Test WER: 1.4067 🎉


In [26]:
predictions

['akajerekani ketereze aasi kutaka kepfundi tieneza kepfundi tiju mufundiko u nyeru na akajerekani klasa na u nyeru akajerekani karimo amata',
 'uwurijobu kweishira hakureishishge ikorana wuhanga kubi ita kubi ikuza kuhiriza ama faranga ukureishishge telefonu jendan',
 'hano hari awa nu batatu babiri muri vo na vijitsi nagavo undi ume usigaye nu vijitsi nagori bicha iku newe zivarariji ume eru ndete imbere ya vo barikungwa amata mwira huri',
 'aba anu benshi wa gaze imbere iinyu baku fite ya maranji yumu tuku nu mngeru ya anditseo ama gamba rimu rimu ligu icho njereza harimu ngibara li jumu tuku aba gawa warimo wa mbae imienda ita ndokanya rabamba ya mashati yu mngeru dete na mapano yumu kaina mbae ama kote na klavate hainu wa mbae ingofero na rinete',
 'anoheza haftisukumdece na machombe wasoche rao karuhu kari amazmeza kwerikomu imetafatru ndetze na sporozi ruhura umuliza',
 'aba anu watuara ama pichi pichi pichumunga wa heze abagode kuma pichi pichi yao vose waagaze iruhande wa hind

In [31]:

# ===================================================================================
# --- Step 6 (Optional): Save Results to a File ---
# ===================================================================================
print("Saving results to 'test_predictions.csv'...")
results_df = pd.DataFrame({
    "Reference": references[:32],
    "Prediction": predictions
})
results_df["wer"] = results_df.apply(
    lambda row: wer_metric.compute(predictions=[row.Prediction], references=[row.Reference]), axis=1
)

# results_df.to_csv("test_predictions_6k_steps__.csv", index=False)
print("✅ Results saved.")

Saving results to 'test_predictions.csv'...
✅ Results saved.


In [16]:
results_df.sort_values(by="wer", ascending=True, inplace=True)

In [32]:
results_df

,Reference,Prediction,wer
0,Akajerekani gateretse hasi ku butaka gapfundik...,akajerekani ketereze aasi kutaka kepfundi tien...,1.125000
1,Uburyo bwo kwishyura hakoreshejwe ikoranabuhan...,uwurijobu kweishira hakureishishge ikorana wuh...,1.250000
2,Hano hari abantu batatu babiri muri bo ni ab'i...,hano hari awa nu batatu babiri muri vo na viji...,0.793103
3,Abantu benshi bahagaze imbere y'inyubako ifite...,aba anu benshi wa gaze imbere iinyu baku fite ...,1.292683
4,Ahantu heza hafite isuku ndetse n'amacumbi was...,anoheza haftisukumdece na machombe wasoche rao...,0.944444
5,Abantu batwara amapikipiki by'umwuga bahetse a...,aba anu watuara ama pichi pichi pichumunga wa ...,1.421053
6,"Abantu benshi cyane, bafite uruhu rwera, bamba...",awano vwenti chani wafite urvurguwela wambaye ...,0.954545
7,"Abagabo babiri bicaye ahantu hamwe, umwe yamba...",aba gawa wabilibicha ya hanu hamge umge ya amb...,0.964286
8,Inshange ya emutiyeni irimo uburyo bwo kwishyu...,mshange ya mtn ilimu uulijewa kushira mafarang...,1.000000
9,Waba ufite amafaranga y'amanyamahanga? Ni byiz...,mfita mafranga ya maja mahanga nibijiza kukana...,1.000000


In [28]:
results_df

,Reference,Prediction,wer
0,Akajerekani gateretse hasi ku butaka gapfundik...,akajerekani ketereze aasi kutaka kepfundi tien...,1.125000
1,Uburyo bwo kwishyura hakoreshejwe ikoranabuhan...,uwurijobu kweishira hakureishishge ikorana wuh...,1.250000
2,Hano hari abantu batatu babiri muri bo ni ab'i...,hano hari awa nu batatu babiri muri vo na viji...,0.793103
3,Abantu benshi bahagaze imbere y'inyubako ifite...,aba anu benshi wa gaze imbere iinyu baku fite ...,1.292683
4,Ahantu heza hafite isuku ndetse n'amacumbi was...,anoheza haftisukumdece na machombe wasoche rao...,0.944444
5,Abantu batwara amapikipiki by'umwuga bahetse a...,aba anu watuara ama pichi pichi pichumunga wa ...,1.421053
6,"Abantu benshi cyane, bafite uruhu rwera, bamba...",awano vwenti chani wafite urvurguwela wambaye ...,0.954545
7,"Abagabo babiri bicaye ahantu hamwe, umwe yamba...",aba gawa wabilibicha ya hanu hamge umge ya amb...,0.964286
8,Inshange ya emutiyeni irimo uburyo bwo kwishyu...,mshange ya mtn ilimu uulijewa kushira mafarang...,1.000000
9,Waba ufite amafaranga y'amanyamahanga? Ni byiz...,mfita mafranga ya maja mahanga nibijiza kukana...,1.000000
